# 광고 클릭율 데이터 분석

## 데이터 불러오기
- kaggle의 Click-Through Rate Prediction 데이터 사용

In [4]:
import pandas as pd

data = pd.read_csv('data/ads_data.csv')
# print(data)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  10000 non-null  float64
 1   Age                       10000 non-null  float64
 2   Area Income               10000 non-null  float64
 3   Daily Internet Usage      10000 non-null  float64
 4   Ad Topic Line             10000 non-null  object 
 5   City                      10000 non-null  object 
 6   Gender                    10000 non-null  object 
 7   Country                   10000 non-null  object 
 8   Timestamp                 10000 non-null  object 
 9   Clicked on Ad             10000 non-null  int64  
dtypes: float64(4), int64(1), object(5)
memory usage: 781.4+ KB
None


## 데이터 설명
|컬럼|내용|
|------------------------|------------------------------------|
|Daily Time Spent on Site|사용자가 사이트에서 보낸 시간 (분)|
|Age|사용자 나이|
|Area Income|사용자가 살고 있는 지역의 평균 수입|
|Daily Internet Usage|하루 인터넷 사용 시간 (분)|
|Ad Topic Line|광고 제목|
|City|사용자 거주 도시|
|Gender|성별|
|Country|사용자 국가|
|Timestamp|광고 노출 시간|
|Clicked on Ad|광고 클릭 여부 (0 : 클릭 안함, 1 : 클릭)|

## 데이터 정리
- Timestamp를 datetime 형태로 변환

In [5]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

In [7]:
print(data.head())

   Daily Time Spent on Site   Age  Area Income  Daily Internet Usage  \
0                     62.26  32.0     69481.85                172.83   
1                     41.73  31.0     61840.26                207.17   
2                     44.40  30.0     57877.15                172.83   
3                     59.88  28.0     56180.93                207.17   
4                     49.21  30.0     54324.73                201.58   

                         Ad Topic Line             City  Gender  \
0      Decentralized real-time circuit         Lisafort    Male   
1       Optional full-range projection  West Angelabury    Male   
2  Total 5thgeneration standardization        Reyesfurt  Female   
3          Balanced empowering success      New Michael  Female   
4  Total 5thgeneration standardization     West Richard  Female   

                        Country           Timestamp  Clicked on Ad  
0  Svalbard & Jan Mayen Islands 2016-06-09 21:43:05              0  
1                     Sing

## A/B Test

### 1. 성별에 따른 광고 클릭률
- A 그룹 : 남성 사용자
- B 그룹 : 여성 사용자
- 비교 지표 : 광고 클릭률(Clicked on Ad)

**가설**
- 귀무가설 : 성별에 따라 광고 클릭률에 차이가 없다
- 대립가설 : 성별에 따라 광고 클릭률에 차이가 있다

**가설 설정 이유**
- 성별이 광고 클릭에 영향을 미치는지 확인

In [12]:
male = data[data['Gender'] == 'Male']
female = data[data['Gender'] == 'Female']

male_click_rate = male['Clicked on Ad'].mean()
female_click_rate = female['Clicked on Ad'].mean()
# print(male_click_rate, female_click_rate)

# t-test 수행 (두 그룹의 클릭률 차이 검정)
from scipy.stats import ttest_ind
ttest, p_value = ttest_ind(male['Clicked on Ad'], female['Clicked on Ad'])
print(f"Male Click Rate: {male_click_rate:.2f}")
print(f"Female Click Rate: {female_click_rate:.2f}")
print(f"T-statistic: {ttest:.2f}, P-value: {p_value:.4f}")

# p_value 가설 해설
if p_value < 0.05:
    print('성별에 따른 클릭률에 통계적으로 유의미한 차이가 있습니다.')
    print('대립가설 : 성별에 따라 광고 클릭률에 차이가 있습니다.')
else:
    print('성별에 따른 클릭률에 통계적으로 유의미한 차이가 없습니다.')
    print('귀무가설 : 성별에 따라 광고 클릭률에 차이가 없습니다.')

Male Click Rate: 0.46
Female Click Rate: 0.51
T-statistic: -4.97, P-value: 0.0000
성별에 따른 클릭률에 통계적으로 유의미한 차이가 있습니다.
대립가설 : 성별에 따라 광고 클릭률에 차이가 있습니다.


#### Cohen's d값 확인

In [14]:
import numpy as np

mean_diff = np.abs(male_click_rate - female_click_rate)
std = np.sqrt(((male['Clicked on Ad'].std() ** 2) + (female['Clicked on Ad'].std() ** 2)) / 2)
cohen_d = mean_diff / std

print(f"Cohen's d: {cohen_d:.2f}")

Cohen's d: 0.10


**통계적으로는 유의미하나, 효과 크기는 0.1로 매우 작다. 따라서 전략에 큰 영향을 미칠 가능성이 매우 적다**

### 2. 연령별 광고 클릭률

**가설**
- 귀무가설 : 연령대별 클릭률 차이 없다
- 대립가설 : 연령대별 클릭률 차이 있다 

In [52]:
from scipy import stats

g1 = data[(data['Age'] < 25)]['Clicked on Ad']
g2 = data[(data['Age'] >= 25) & (data['Age'] < 30)]['Clicked on Ad']
g3 = data[(data['Age'] >= 30) & (data['Age'] < 35)]['Clicked on Ad']
g4 = data[(data['Age'] >= 35) & (data['Age'] < 40)]['Clicked on Ad']
g5 = data[(data['Age'] >= 40) & (data['Age'] < 45)]['Clicked on Ad']
g6 = data[(data['Age'] >= 45)]['Clicked on Ad']

f_stat, p_value = stats.f_oneway(g1, g2, g3, g4, g5, g6)

print(f'stats: {f_stat:.4f}')
print(f'pvalue: {p_value:.4f}')

if p_value < 0.05:
    print('귀무가설 기각 -> 연령대별로 클릭률 차이 있다')
else:
    print('귀무가설 채택 -> 연령대별 클릭률 차이 없다')


stats: 578.1662
pvalue: 0.0000
귀무가설 기각 -> 연령대별로 클릭률 차이 있다


### Tukey's HSD (사후 검정)

In [67]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def age_group(age):
    if age < 25:
        return 'group 1'
    elif age < 30:
        return 'group 2'
    elif age < 35:
        return 'group 3'
    elif age < 40:
        return 'group 4'
    elif age < 45:
        return 'group 5'
    else:
        return 'group 6'

data['Age_Group'] = data['Age'].apply(age_group)

tukey_result = pairwise_tukeyhsd(
    endog = data['Clicked on Ad'],
    groups = data['Age_Group'],
    alpha = 0.05
)

result = tukey_result.summary()
print(result)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper  reject
------------------------------------------------------
group 1 group 2  -0.0599 0.0151 -0.1126 -0.0073   True
group 1 group 3   0.0233 0.7975 -0.0287  0.0753  False
group 1 group 4    0.168    0.0  0.1162  0.2197   True
group 1 group 5   0.4508    0.0  0.3961  0.5055   True
group 1 group 6   0.5703    0.0  0.5174  0.6231   True
group 2 group 3   0.0832    0.0  0.0427  0.1238   True
group 2 group 4   0.2279    0.0  0.1876  0.2681   True
group 2 group 5   0.5108    0.0  0.4668  0.5548   True
group 2 group 6   0.6302    0.0  0.5886  0.6718   True
group 3 group 4   0.1446    0.0  0.1053   0.184   True
group 3 group 5   0.4275    0.0  0.3844  0.4707   True
group 3 group 6    0.547    0.0  0.5062  0.5877   True
group 4 group 5   0.2829    0.0    0.24  0.3257   True
group 4 group 6   0.4023    0.0  0.3619  0.4427   True
group 5 group 6   0.1195    0.0  0.0753  0.1636   True
----------

**group 1 과 group 3 의 reject가 False 이기 때문에 유의미한 차이가 없다. 하지만 다른 모든 것은 유의미한 차이가 있는것을 확인 할 수 있다**